# Get backend information with Qiskit

<details>
<summary><b>Package versions</b></summary>

The code on this page was developed using the following requirements.
We recommend using these versions or newer.

```
qiskit[all]~=1.3.2
qiskit-ibm-runtime~=0.34.0
qiskit-aer~=0.15.1
qiskit-serverless~=0.18.1
qiskit-ibm-catalog~=0.2
qiskit-addon-sqd~=0.8.1
qiskit-addon-utils~=0.1.0
qiskit-addon-mpf~=0.2.0
qiskit-addon-aqc-tensor~=0.1.2
qiskit-addon-obp~=0.1.0
scipy~=1.15.1
pyscf~=2.8.0; sys.platform != 'win32'
```
</details>

This page explains how to use Qiskit to find information about your available backends.

## List backends

To view the backends you have access to, you can either view a list on the [Compute resources page,](https://quantum.ibm.com/services/resources?tab=yours) or you can use the [`QiskitRuntimeService.backends()`](../api/qiskit-ibm-runtime/qiskit-runtime-service#backends) method. This method returns a list of [`IBMBackend`](../api/qiskit-ibm-runtime/ibm-backend) instances:

In [1]:
# Initialize your account
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService(instance="ibm-q/open/main")

service.backends()

[<IBMBackend('ibm_brisbane')>,
 <IBMBackend('ibm_sherbrooke')>,
 <IBMBackend('ibm_kyiv')>]

The [`QiskitRuntimeService.backend()`](../api/qiskit-ibm-runtime/qiskit-runtime-service#backend) method (note that this is singular: *backend*) takes the name of the backend as the input parameter and returns an [`IBMBackend`](../api/qiskit-ibm-runtime/ibm-backend) instance representing that particular backend:

In [2]:
service.backend("ibm_brisbane")

<IBMBackend('ibm_brisbane')>

## Filter backends

You can also filter the available backends by their properties. For more general filters, set the `filters` argument to a function that accepts a backend object and returns `True` if it meets your criteria. Refer to the [API documentation](../api/qiskit-ibm-runtime/qiskit-runtime-service#backends) for more details.

The following code returns only backends that fit these criteria:

*   Are real quantum devices (`simulator=False`)
*   Are currently operational (`operational=True`)
*   Have at least 5 qubits (`min_num_qubits=5`)

In [3]:
service.backends(simulator=False, operational=True, min_num_qubits=5)

[<IBMBackend('ibm_brisbane')>,
 <IBMBackend('ibm_sherbrooke')>,
 <IBMBackend('ibm_kyiv')>]

Use these keyword arguments to filter by any attribute in backend configuration ([JSON schema](https://github.com/Qiskit/ibm-quantum-schemas/blob/main/schemas/backend_configuration_schema.json)) or status ([JSON schema](https://github.com/Qiskit/ibm-quantum-schemas/blob/main/schemas/backend_status_schema.json)). A similar method is [`QiskitRuntimeService.least_busy()`](../api/qiskit-ibm-runtime/qiskit-runtime-service#least_busy), which takes the same filters as `backends()` but returns the backend that matches the filters and has the least number of jobs pending in the queue:

In [4]:
service.least_busy(operational=True, min_num_qubits=5)

<IBMBackend('ibm_kyiv')>

## Static backend information

Some information about a backend does not change regularly, such as its name, version, the number of qubits it has, and the types of features it supports. This information is available as attributes of the `backend` object.

The following cell builds a description of a backend.

In [5]:
backend = service.backend("ibm_sherbrooke")

print(
    f"Name: {backend.name}\n"
    f"Version: {backend.version}\n"
    f"No. of qubits: {backend.num_qubits}\n"
)

Name: ibm_sherbrooke
Version: 2
No. of qubits: 127



For a full list of attributes, see the [`IBMBackend` API documentation](/api/qiskit-ibm-runtime/ibm-backend).

## Dynamic backend information

Backends can also have properties that change whenever the backed is calibrated, such as qubit frequency and operation error rates. Backends are usually calibrated every 24 hours, and their properties update after the calibration sequence completes. These properties can be used when optimizing quantum circuits or to construct noise models for a classical simulator.


### Qubit properties

The `backend.qubit_properties` method returns information about the qubits' physical attributes. This includes the qubit frequency in GHz and decay times (`t1` and `t2`) in µs.

In [6]:
backend.qubit_properties(0)  # properties of qubit 0

QubitProperties(t1=0.0003843410308287623, t2=0.0001311444380168079, frequency=4635662846.425661)

### Instruction properties

The `backend.target` attribute is a `qiskit.transpiler.Target` object: an object that contains all the information needed to transpile a circuit for that backend. This includes instruction errors and durations. For example, the following cell gets the properties for an [`ecr` gate](/api/qiskit/qiskit.circuit.library.ECRGate) acting between qubits 1 and 0.

In [7]:
backend.target["ecr"][(1, 0)]

InstructionProperties(duration=5.333333333333332e-07, error=0.007366632564165032, calibration=PulseQobj)

The following cell shows the properties for a measurement operation (including the readout error) on qubit 0.

In [8]:
backend.target["measure"][(0,)]

InstructionProperties(duration=1.216e-06, error=0.01025390625, calibration=PulseQobj)

## Next steps

<Admonition type="tip" title="Recommendations">
    - Try the [Grover's algorithm](https://learning.quantum.ibm.com/tutorial/grovers-algorithm) tutorial.
    - Review the [QiskitRuntime backend API](/api/qiskit-ibm-runtime/qiskit-runtime-service#backend) reference.
</Admonition>